In [3]:
import numpy as np
import random

# Определяем дороги (ребра) и их исходные характеристики
roads = {
    'A': {'base_time': 2, 'traffic_factor': 0.01, 'quality': 1.0, 'cost': 10, 'comfort': 0.8},
    'B': {'base_time': 3, 'traffic_factor': 0.015, 'quality': 0.8, 'cost': 12, 'comfort': 0.9},
    'C': {'base_time': 5, 'traffic_factor': 0.02, 'quality': 1.2, 'cost': 15, 'comfort': 1.0},
    'D': {'base_time': 4, 'traffic_factor': 0.012, 'quality': 1.1, 'cost': 8, 'comfort': 0.7},
    'E': {'base_time': 6, 'traffic_factor': 0.025, 'quality': 0.9, 'cost': 20, 'comfort': 0.85}
}

# Количество водителей и распределение трафика
traffic_distribution = {
    'A': 30,
    'B': 20,
    'C': 50,
    'D': 40,
    'E': 10
}

total_traffic = sum(traffic_distribution.values())

# Параметры водителей: весовые коэффициенты для факторов времени, стоимости и комфорта
drivers = {
    'weights': {
        'time': 0.5,  # Важность времени
        'cost': 0.3,  # Важность стоимости
        'comfort': 0.2  # Важность комфорта
    }
}

# Функция для расчета общего "удовольствия" водителей от маршрута с учетом всех факторов
def route_score(base_time, traffic, traffic_factor, quality, cost, comfort, driver_weights):
    time_component = base_time + traffic_factor * traffic / quality
    cost_component = cost
    comfort_component = comfort
    return (driver_weights['time'] * time_component +
            driver_weights['cost'] * cost_component +
            driver_weights['comfort'] * comfort_component)

# Функция для учета аварий
def road_event(road, event_probability=0.1):
    event_type = random.choices(['none', 'partial', 'full'], weights=[0.8, 0.15, 0.05])[0]
    if event_type == 'partial':
        print(f"Произошло замедление на дороге {road} (частичная авария).")
        return 1.5  # Время увеличивается на 50%
    elif event_type == 'full':
        print(f"Произошло полное закрытие дороги {road} (серьезная авария).")
        return float('inf')  # Дорога полностью закрыта
    return 1.0  # Нет аварий

# Функция для перераспределения трафика
def redistribute_traffic(traffic_distribution, roads, drivers):
    new_distribution = traffic_distribution.copy()

    # Рассчитываем "удовольствие" для каждой дороги
    scores = {}
    for road, props in roads.items():
        event_multiplier = road_event(road)
        if event_multiplier != float('inf'):
            scores[road] = route_score(
                props['base_time'] * event_multiplier,
                traffic_distribution[road],
                props['traffic_factor'],
                props['quality'],
                props['cost'],
                props['comfort'],
                drivers['weights']
            )
        else:
            scores[road] = float('inf')  # Если дорога закрыта, она недоступна

    # Перераспределяем трафик на основе оценки маршрутов
    for road in roads:
        best_score = min(scores.values())
        if scores[road] > best_score and scores[road] != float('inf'):
            traffic_shift = (scores[road] - best_score) * 0.1
            shift = min(traffic_shift, traffic_distribution[road])
            for target_road in roads:
                if scores[target_road] == best_score:
                    new_distribution[road] -= shift
                    new_distribution[target_road] += shift

    return new_distribution

# Q-Learning параметры
q_table = np.zeros((len(roads), 5))  # Таблица Q-значений (дороги х действия)
alpha = 0.1  # Коэффициент обучения
gamma = 0.9  # Коэффициент дисконтирования
epsilon = 0.1  # Вероятность случайного выбора действия

# Функция для выбора действия с использованием Q-Learning
def choose_action(road_index, scores):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(5))  # Случайное действие
    return np.argmax(q_table[road_index])  # Действие с максимальным Q-значением

# Обновление Q-значений
def update_q_table(road_index, action, reward, next_road_index):
    best_future_q = np.max(q_table[next_road_index])
    current_q = q_table[road_index][action]
    new_q = current_q + alpha * (reward + gamma * best_future_q - current_q)
    q_table[road_index][action] = new_q

# Функция для расчета награды на основе маршрута
def calculate_reward(score):
    return -score  # Чем лучше оценка маршрута, тем выше награда

# Оценка достижения равновесия Уордаропа
def check_equilibrium(old_distribution, new_distribution, tolerance=0.01):
    changes = [abs(old_distribution[road] - new_distribution[road]) for road in old_distribution]
    return max(changes) < tolerance

# Симуляция для нескольких раундов
num_rounds = 20
equilibrium_reached = False
for round_num in range(1, num_rounds + 1):
    print(f"\nРаунд {round_num}:")
    print("Текущее распределение трафика:", traffic_distribution)

    old_distribution = traffic_distribution.copy()

    # Перераспределение трафика с помощью Q-learning
    for road_index, road in enumerate(roads):
        current_score = route_score(
            roads[road]['base_time'], traffic_distribution[road],
            roads[road]['traffic_factor'], roads[road]['quality'],
            roads[road]['cost'], roads[road]['comfort'], drivers['weights']
        )
        action = choose_action(road_index, current_score)
        new_distribution = redistribute_traffic(traffic_distribution, roads, drivers)

        # Рассчитываем новую оценку маршрута и награду
        new_score = route_score(
            roads[road]['base_time'], new_distribution[road],
            roads[road]['traffic_factor'], roads[road]['quality'],
            roads[road]['cost'], roads[road]['comfort'], drivers['weights']
        )
        reward = calculate_reward(new_score)
        next_road_index = road_index

        # Обновление Q-значений
        update_q_table(road_index, action, reward, next_road_index)

    # Вывод результатов текущего раунда
    for road, props in roads.items():
        time = travel_time(
            props['base_time'], traffic_distribution[road],
            props['traffic_factor'], props['quality']
        )
        print(f"Дорога {road}: оценка маршрута {time:.2f}")

    # Проверка достижения равновесия
    equilibrium_reached = check_equilibrium(old_distribution, traffic_distribution)
    if equilibrium_reached:
        print(f"Равновесие Уордаропа достигнуто на {round_num}-м раунде.")
        break

    # Адаптация распределения трафика после каждого раунда
    traffic_distribution = redistribute_traffic(traffic_distribution, roads, drivers)

# Итоговое распределение трафика
print("\nИтоговое распределение трафика:", traffic_distribution)



Раунд 1:
Текущее распределение трафика: {'A': 30, 'B': 20, 'C': 50, 'D': 40, 'E': 10}
Произошло замедление на дороге D (частичная авария).
Произошло полное закрытие дороги E (серьезная авария).
Произошло полное закрытие дороги B (серьезная авария).
Произошло полное закрытие дороги A (серьезная авария).
Дорога A: оценка маршрута 2.30
Дорога B: оценка маршрута 3.38
Дорога C: оценка маршрута 5.83
Дорога D: оценка маршрута 4.44
Дорога E: оценка маршрута 6.28
Равновесие Уордаропа достигнуто на 1-м раунде.

Итоговое распределение трафика: {'A': 30, 'B': 20, 'C': 50, 'D': 40, 'E': 10}
